## CUSTOMER LIFETIME VALUE (Müşteri Yaşam Boyu Değeri)

In [ ]:
# Değişkenler
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.

- Burada amacımız 'Customer Lifetime Value'ları üzerinden müşteri segmentasyonu yapmaktır.

### 1. Data Preparation(Veri Hazırlama):

In [1]:
#import datetime as dt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


pd.set_option('display.max_columns', None) # bütün sütunları göstermek için ayar
# pd.set_option('display.max_rows', None) # bütün sütunları göstermek için ayar(opsiyonel)
pd.set_option('display.float_format', lambda x: '%.5f' % x) #float sayıların virgülden sonra kaç hanesi gösterilmeli onu ayarlıyor

In [2]:
df_ = pd.read_excel("C:\\Users\\Monster\\Desktop\\dataa\\online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy() #veri setinin ilk okunduğu halini kopyalar
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [3]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

- ID'sini bilmediğimiz müşterilerle ilgili işlem yapamayacağımız için veri setinden atarız eksik değerleri(NaN value).

In [4]:
# Başında C ifadesi bulunan "Invoice" değerlerini de veri setinden atmamız gerekir
df = df[~df["Invoice"].str.contains("C", na=False)]

In [5]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95000,17530.00000,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom


In [6]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,515255.00000,10.95663,-9600.00000,1.00000,3.00000,10.00000,19152.00000,104.35401
InvoiceDate,515255,2010-06-28 17:41:06.558422528,2009-12-01 07:45:00,2010-03-21 13:27:00,2010-07-06 13:13:00,2010-10-15 14:27:00,2010-12-09 20:01:00,NaN
Price,515255.00000,3.95637,-53594.36000,1.25000,2.10000,4.21000,25111.09000,127.68856
Customer ID,407695.00000,15368.50411,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000,1679.79570


In [7]:
# quantity değerlerinin istatistiklerinde tutarsızlık var negatif gelmemeliydi bu nedenle quantity değerleri 0'dan büyük olanları alıyoruz

df = df[(df['Quantity'] > 0)]

In [8]:
df.dropna(inplace=True)

C:\Users\Monster\AppData\Local\Temp\ipykernel_9936\1379821321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [9]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,407695.00000,13.58669,1.00000,2.00000,5.00000,12.00000,19152.00000,96.84223
InvoiceDate,407695,2010-07-01 10:10:10.782177792,2009-12-01 07:45:00,2010-03-26 14:01:00,2010-07-09 15:46:00,2010-10-14 17:09:00,2010-12-09 20:01:00,NaN
Price,407695.00000,3.29419,0.00000,1.25000,1.95000,3.75000,10953.50000,34.75666
Customer ID,407695.00000,15368.50411,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000,1679.79570


In [10]:
# veri setinde quantity ve price değerleri ayrı ayrı verilmiş ancak TotalPrice değeri verilmemiş bu nedenle onu hesaplayıp veri setine yeni
# bir feature ya da column olarak eklememiz gerekmektedir.

df["TotalPrice"] = df["Quantity"] * df["Price"]

C:\Users\Monster\AppData\Local\Temp\ipykernel_9936\377486096.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TotalPrice"] = df["Quantity"] * df["Price"]


In [11]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [12]:
# müşteri değerini hesaplamak için müşteri özelinde Total Price'lara sahip olmamız, hesaplamamız gerekmektedir
# ilk olarak verileri Customer ID'lere göre groupby'a alıyoruz yani grupluyoruz
# sonra 'Invoice': lambda x: x.nunique() ile Invoice değişkeninin nunique'ini alıyoruz böylece her bir müşterinin eşsiz kaç faturası var 
# onu görürüz ve total transaction değerine erişmiş oluruz yani müşteri işlem(alışveriş) sayısı
# 'TotalPrice': lambda x: x.sum() aynı customer id'ye sahip olanların totalprice'larının toplamını alır

cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})

In [13]:
cltv_c 

,Invoice,Quantity,TotalPrice
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000
...,...,...,...
18283.00000,6,336,641.77000
18284.00000,1,494,461.68000
18285.00000,1,145,427.00000


In [14]:
# sütunları istediğimiz şekilde, daha anlamlı hale getirecek biçimde isimlendiririz
cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

In [15]:
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


### 2. Average Order Value (average_order_value = total_price / total_transaction)

In [16]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

In [17]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.00000,11,70,372.86000,33.89636
12347.00000,2,828,1323.32000,661.66000
12348.00000,1,373,222.16000,222.16000
12349.00000,3,993,2671.14000,890.38000
12351.00000,1,261,300.93000,300.93000


### 3. Purchase Frequency (total_transaction / total_number_of_customers)

In [18]:
# cltv_c.shape ifadesi veri setinin boyutunu verir yani kaç satır kaç sütun olduğunu
# cltv_c.shape[0] ise satır sayısını verir yani toplam müşteri sayısını
cltv_c.shape[0]

cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

In [19]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255
12347.00000,2,828,1323.32000,661.66000,0.00046
12348.00000,1,373,222.16000,222.16000,0.00023
12349.00000,3,993,2671.14000,890.38000,0.00070
12351.00000,1,261,300.93000,300.93000,0.00023


### 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)

In [20]:
cltv_c[cltv_c["total_transaction"] > 1]

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255
12347.00000,2,828,1323.32000,661.66000,0.00046
12349.00000,3,993,2671.14000,890.38000,0.00070
12352.00000,2,188,343.80000,171.90000,0.00046
12356.00000,3,1826,3562.25000,1187.41667,0.00070
...,...,...,...,...,...
18276.00000,5,1060,1320.66000,264.13200,0.00116
18277.00000,4,381,1069.67000,267.41750,0.00093
18283.00000,6,336,641.77000,106.96167,0.00139


In [21]:
cltv_c[cltv_c["total_transaction"] > 1].shape[0]

2893

In [22]:
# birden fazla alışveriş yapanların sayısı lazım old. için "cltv_c[cltv_c["total_transaction"] > 1].shape[0]" ile sayısını alırız 

repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]

In [23]:
repeat_rate

0.6706073249884098

In [24]:
churn_rate = 1 - repeat_rate
churn_rate

0.3293926750115902

### 5. Profit Margin (profit_margin =  total_price * 0.10)

In [25]:
# "0.10" sabit bir değerdir, şirket tarafından belirlenmesi gereken sabit bir değer

cltv_c['profit_margin'] = cltv_c['total_price'] * 0.10

In [26]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300


### 6. Customer Value (customer_value = average_order_value * purchase_frequency)

In [27]:
cltv_c['customer_value'] = cltv_c['average_order_value'] * cltv_c["purchase_frequency"]

In [28]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
Customer ID,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976


### 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)

In [29]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

In [30]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290


In [31]:
# müşteri yaşam boyu değeri en yüksek yani bizim için en değerli müşterileri görmek için büyükten küçüğe sıralama yapılır
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


In [32]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
total_unit,4314.00000,1284.01113,6458.45205,1.00000,158.00000,382.00000,995.25000,220600.00000
total_price,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000
average_order_value,4314.00000,378.14723,492.51721,0.00000,181.95632,286.94600,423.53375,11880.84000
purchase_frequency,4314.00000,0.00103,0.00189,0.00023,0.00023,0.00046,0.00116,0.04752
profit_margin,4314.00000,204.72887,891.25232,0.00000,30.79500,70.55500,172.28025,34916.43500
customer_value,4314.00000,0.47457,2.06595,0.00000,0.07138,0.16355,0.39935,80.93749
cltv,4314.00000,5883.60651,156068.63594,0.00000,6.67369,35.03173,208.87042,8579573.77276


### 8. Creating Segment(Segmentlerin Oluşturulması)

In [33]:
cltv_c.sort_values(by="cltv", ascending=False).tail()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061
14103.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000
14827.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000


In [34]:
# cltv değerlerine göre küçükten büyüğe doğru sıralar ve bunları dört parçaya(quartile) ayırır ardından da etiketleri atar
# en küçük "cltv" değerlerine karşılık "D" label'ı gelir, en büyükleri içinse "A" label'ı gelir
# yani en yüksek cltv değerlerine sahip olanlar A sınıfı müşteriler diyebiliriz

cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

In [35]:
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276,A
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829,A
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615,A
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978,A
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003,A


In [36]:
cltv_c.groupby("segment").agg({"count", "mean", "sum"})

C:\Users\Monster\AppData\Local\Temp\ipykernel_9936\2793452465.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cltv_c.groupby("segment").agg({"count", "mean", "sum"})


total_transaction              total_unit                total_price  \
                     mean    sum count       mean      sum count        mean   
segment                                                                        
D                 1.22892   1326  1079  109.00463   117616  1079   178.18826   
C                 2.00371   2160  1078  283.05659   305135  1078   475.89652   
B                 3.76902   4063  1078  680.15863   733211  1078  1131.35918   
A                10.81186  11666  1079 4062.33735  4383262  1079  6401.40546   

                            average_order_value                     \
                  sum count                mean          sum count   
segment                                                              
D        192265.13000  1079           157.37300 169805.46550  1079   
C        513016.45300  1078           294.45487 317422.35514  1078   
B       1219605.20000  1078           389.72832 420127.12365  1078   
A       6907116.49100  1079           670.96592 723972.22347  1079   

        purchase_frequency               profit_margin                     \
                      mean     sum count          mean          sum count   
segment                                                                     
D                  0.00028 0.30737  1079      17.81883  19226.51300  1079   
C                  0.00046 0.50070  1078      47.58965  51301.64530  1078   
B                  0.00087 0.94182  1078     113.13592 121960.52000  1078   
A                  0.00251 2.70422  1079     640.14055 690711.64910  1079   

        customer_value                         cltv                       
                  mean        sum count        mean            sum count  
segment                                                                   
D              0.04130   44.56772  1079     2.64071     2849.32790  1079  
C              0.11031  118.91897  1078    16.86852    18184.26342  1078  
B              0.26225  282.70867  1078    96.05739   103549.86329  1078  
A              1.48387 1601.09330  1079 23408.05843 25257295.04118  1079

In [37]:
cltv_c.to_csv("cltc_c.csv")

### Functionalization(Tüm Sürecin Fonksiyonlaştırılması)

In [38]:
def create_cltv_c(dataframe, profit=0.10):

    # Veriyi hazırlama
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[(dataframe['Quantity'] > 0)]
    dataframe.dropna(inplace=True)
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                                   'Quantity': lambda x: x.sum(),
                                                   'TotalPrice': lambda x: x.sum()})
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    
    # avg_order_value
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    
    # purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    
    # repeat rate & churn rate
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    
    # profit_margin
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    
    # Customer Value
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    
    # Customer Lifetime Value
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    
    # Segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_c

In [39]:
# dataframe'i en baştaki haline dönmesi için sıfırlıyoruz yani en başta kopyasını oluşturmuştuk onu kullanıyoruz
df = df_.copy()

In [40]:
clv = create_cltv_c(df)

In [41]:
clv

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357,C
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546,B
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326,D
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041,A
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290,D
...,...,...,...,...,...,...,...,...,...
18283.00000,6,336,641.77000,106.96167,0.00139,64.17700,0.14876,28.98443,C
18284.00000,1,494,461.68000,461.68000,0.00023,46.16800,0.10702,14.99989,C
18285.00000,1,145,427.00000,427.00000,0.00023,42.70000,0.09898,12.83103,C
